In [1]:
import os
import yaml
import glob

In [2]:
!nvidia-smi

Wed Jun  2 09:33:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%env DRIVE_FOLDER=/content/drive/MyDrive
%env PROJECT_FOLDER=/content/drive/MyDrive/ProgettoML
%env SSD_FOLDER=/content/drive/MyDrive/ProgettoML/SSD
%env CYCLEGAN_FOLDER=/content/drive/MyDrive/ProgettoML/CycleGAN
%env ADAIN_FOLDER=/content/drive/MyDrive/ProgettoML/AdaIN
%env VOC_ROOT=/content/drive/MyDrive/ProgettoML/datasets/VOC
%env CLIPART_ROOT=/content/drive/MyDrive/ProgettoML/datasets/CLIPART

env: DRIVE_FOLDER=/content/drive/MyDrive
env: PROJECT_FOLDER=/content/drive/MyDrive/ProgettoML
env: SSD_FOLDER=/content/drive/MyDrive/ProgettoML/SSD
env: CYCLEGAN_FOLDER=/content/drive/MyDrive/ProgettoML/CycleGAN
env: ADAIN_FOLDER=/content/drive/MyDrive/ProgettoML/AdaIN
env: VOC_ROOT=/content/drive/MyDrive/ProgettoML/datasets/VOC
env: CLIPART_ROOT=/content/drive/MyDrive/ProgettoML/datasets/CLIPART


#<h1>Baseline SSD</h1>

In [7]:
os.chdir("/content")

!git clone https://github.com/ekoops/SSD.git
if not os.path.isdir(os.environ["SSD_FOLDER"]):
  !mv SSD $SSD_FOLDER
else:
  !rsync -a ./SSD/ $SSD_FOLDER/
  !rm -r SSD

!pip install -r $SSD_FOLDER/requirements.txt

Cloning into 'SSD'...
remote: Enumerating objects: 1239, done.
remote: Counting objects: 100% (319/319), done.
remote: Compressing objects: 100% (240/240), done.
remote: Total 1239 (delta 202), reused 185 (delta 78), pack-reused 920
Receiving objects: 100% (1239/1239), 838.27 KiB | 2.93 MiB/s, done.
Resolving deltas: 100% (737/737), done.


##<h3>Baseline SSD training</h3>

In [ ]:
os.chdir(os.environ["SSD_FOLDER"])
!python train.py --config-file configs/vgg_ssd300_voc0712.yaml

Traceback (most recent call last):
  File "train.py", line 5, in <module>
    import torch
  File "/usr/local/lib/python3.7/dist-packages/torch/__init__.py", line 196, in <module>
    from torch._C import *
RuntimeError: KeyboardInterrupt: 


##<h3>Baseline SSD test on ClipArt1k dataset

In [ ]:
os.chdir(os.environ["SSD_FOLDER"])
!python test.py --config-file configs/vgg_ssd300_clipart.yaml

Traceback (most recent call last):
  File "<frozen importlib._bootstrap_external>", line 818, in get_code
  File "<frozen importlib._bootstrap_external>", line 916, in get_data
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ProgettoML/SSD/ssd/__pycache__/__init__.cpython-37.pyc'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "test.py", line 8, in <module>
    from ssd.config import cfg
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 967, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 677, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 818, in get_code
KeyboardInterrupt


#<h1>CycleGAN</h1>

In [ ]:
os.chdir("/content")
!git clone https://github.com/ekoops/pytorch-CycleGAN-and-pix2pix.git
if not os.path.isdir(os.environ["CYCLEGAN_FOLDER"]):
  !mv pytorch-CycleGAN-and-pix2pix $CYCLEGAN_FOLDER
else:
  !rsync -a ./pytorch-CycleGAN-and-pix2pix/ $CYCLEGAN_FOLDER/
  !rm -r pytorch-CycleGAN-and-pix2pix

!pip install -r $CYCLEGAN_FOLDER/requirements.txt

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 2360 (delta 3), reused 8 (delta 3), pack-reused 2349
Receiving objects: 100% (2360/2360), 8.10 MiB | 40.26 MiB/s, done.
Resolving deltas: 100% (1511/1511), done.


##<h3>CycleGAN training</h3>

In [ ]:
%env trainA_FILTER_FILE=/content/drive/MyDrive/ProgettoML/datasets/VOC_trainval_merge.txt
%env trainB_FILTER_FILE=/content/drive/MyDrive/ProgettoML/datasets/CLIPART/clipart/ImageSets/Main/train.txt
os.chdir(os.environ["CYCLEGAN_FOLDER"])
!python train.py --name VOC0712_ClipArt1k --dataroot inputs --display_id -1\
  --verbose --save_latest_freq 500000 --save_epoch_freq 1 --epoch_count 19\
  --n_epochs 10 --n_epochs_decay 10 --lr 0.00001 --continue_train

env: trainA_FILTER_FILE=/content/drive/MyDrive/ProgettoML/datasets/VOC_trainval_merge.txt
env: trainB_FILTER_FILE=/content/drive/MyDrive/ProgettoML/datasets/CLIPART/clipart/ImageSets/Main/train.txt
----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: True                          	[default: False]
                crop_size: 256                           
                 dataroot: inputs                        	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            	[default: 1]
            display_ncols: 4                             
             displ

##<h3>VOC 2007/12 trainval datasets DT to ClipArt1k domain</h3>

In [ ]:
%env testA_FILTER_FILE=/content/drive/MyDrive/ProgettoML/datasets/VOC_trainval_merge.txt
os.chdir(os.environ["CYCLEGAN_FOLDER"])
!python test.py --name VOC0712_ClipArt1k --model test --dataroot inputs/testA\
  --preprocess none --verbose --no_dropout --num_test 16551 --results_dir\
  ../datasets/ --no_real

env: testA_FILTER_FILE=/content/drive/MyDrive/ProgettoML/datasets/VOC_trainval_merge.txt
----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: inputs/testA                  	[default: None]
             dataset_mode: single                        
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False         

In [ ]:
os.chdir("/content/drive/MyDrive/ProgettoML/datasets/VOC0712_ClipArt1k")
!ls -l VOC2012/JPEGImages/ | wc -l

11541


# Baseline SSD fine-tuning with VOC 2007/12-to-ClipArt1k images

In [8]:
%env VOC_ROOT=/content/drive/MyDrive/ProgettoML/datasets/VOC0712_ClipArt1k
%env CLIPART_ROOT=/content/drive/MyDrive/ProgettoML/datasets/CLIPART

GUARD = True
startIndex = 3

os.chdir(os.environ["SSD_FOLDER"])
configFileList = [
        "vgg_ssd300_voc0712toclipart_ft", #It: 520
        "vgg_ssd300_voc0712toclipart_ft2", #It: 1100
        "vgg_ssd300_voc0712toclipart_ft3", #It: 10000
]

lastCheckpointFilePath = "outputs/vgg_ssd300_voc0712/last_checkpoint.txt"

if not GUARD:
  for configFile in configFileList:
    folderPath = f"outputs/{configFile}"
    !rm -r $folderPath
    !mkdir $folderPath
    !cp $lastCheckpointFilePath $folderPath/

for idx, configFile in enumerate(configFileList, 1):
  configFilePath = f"configs/{configFile}.yaml"
  if idx >= startIndex:
    !python train.py --config-file $configFilePath

env: VOC_ROOT=/content/drive/MyDrive/ProgettoML/datasets/VOC0712_ClipArt1k
env: CLIPART_ROOT=/content/drive/MyDrive/ProgettoML/datasets/CLIPART
2021-06-02 09:37:41,153 SSD INFO: Using 1 GPUs
2021-06-02 09:37:41,154 SSD INFO: Namespace(config_file='configs/vgg_ssd300_voc0712toclipart_ft3.yaml', distributed=False, enable_style_transfer=False, eval_step=2500, local_rank=0, log_step=10, num_gpus=1, opts=[], save_step=2500, skip_test=False, use_tensorboard=True)
2021-06-02 09:37:41,154 SSD INFO: Loaded configuration file configs/vgg_ssd300_voc0712toclipart_ft3.yaml
2021-06-02 09:37:41,155 SSD INFO: 
MODEL:
  NUM_CLASSES: 21
INPUT:
  IMAGE_SIZE: 300
DATASETS:
  TRAIN: ("voc_2007_trainval", "voc_2012_trainval")
  TEST: ("clipart_test", )
SOLVER:
  MAX_ITER: 130000
  LR_STEPS: []
  GAMMA: 0.1
  BATCH_SIZE: 32
  LR: 1e-5

OUTPUT_DIR: 'outputs/vgg_ssd300_voc0712toclipart_ft3'
2021-06-02 09:37:41,155 SSD INFO: Running with config:
ADAIN:
  DATASETS:
    STYLE: ()
  IMPL_FOLDER: 
  INPUT:
    PRES

# AdaIN

In [ ]:
os.chdir("/content")

!git clone https://github.com/ekoops/pytorch-AdaIN.git
if not os.path.isdir(os.environ["ADAIN_FOLDER"]):
  !mv pytorch-AdaIN $ADAIN_FOLDER
else:
  !rsync -a ./pytorch-AdaIN/ $ADAIN_FOLDER/
  !rm -r pytorch-AdaIN

!pip install -r $ADAIN_FOLDER/requirements.txt

Cloning into 'pytorch-AdaIN'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 299 (delta 5), reused 8 (delta 3), pack-reused 286
Receiving objects: 100% (299/299), 7.78 MiB | 44.77 MiB/s, done.
Resolving deltas: 100% (146/146), done.
     |████████████████████████████████| 20.3MB 1.3MB/s 
     |████████████████████████████████| 2.2MB 47.4MB/s 
ERROR: Could not find a version that satisfies the requirement pkg-resources==0.0.0 (from -r /content/drive/MyDrive/ProgettoML/AdaIN/requirements.txt (line 3)) (from versions: none)
ERROR: No matching distribution found for pkg-resources==0.0.0 (from -r /content/drive/MyDrive/ProgettoML/AdaIN/requirements.txt (line 3))


# Baseline SSD fine-tuning with VOC 2007/12 using AdaIN style transferring

### Validation

In [ ]:
%env VOC_ROOT=/content/drive/MyDrive/ProgettoML/datasets/VOC
%env CLIPART_ROOT=/content/drive/MyDrive/ProgettoML/datasets/CLIPART

GUARD = True
startIndex = 1

os.chdir(os.environ["SSD_FOLDER"])
configFileList = [
        "vgg_ssd300_voc0712_AdaINst_ft", #TR: 1.0, alpha: 1.0
        "vgg_ssd300_voc0712_AdaINst_ft2", #TR: 0.5, alpha: 1.0
        "vgg_ssd300_voc0712_AdaINst_ft3", #TR: 1.0, alpha: 0.5
        "vgg_ssd300_voc0712_AdaINst_ft4" #TR: 0.5, alpha: 0.5
]
lastCheckpointFilePath = "outputs/vgg_ssd300_voc0712/last_checkpoint.txt"

if not GUARD:
  for configFile in configFileList:
    folderPath = f"outputs/{configFile}"
    !rm -r $folderPath
    !mkdir $folderPath
    !cp $lastCheckpointFilePath $folderPath/

bestConfig = {
  "configFile": "",
  "mAP": 0.0,
}

for idx, configFile in enumerate(configFileList, 1):
  configFilePath = f"configs/{configFile}.yaml"
  if idx >= startIndex:
    !python train.py --config-file $configFilePath --enable_style_transfer

for configFile in configFileList:
  path = f"outputs/{configFile}/inference/clipart_test/*.txt"
  for filename in glob.glob(path):
    with open(filename, 'r') as f:
      line = f.read()
      mAP = float(line.split()[1])
      if mAP > float(bestConfig["mAP"]):
        bestConfig = {
          "configFile": configFile,
          "mAP": mAP
        }

bestConfigFilePath = f"configs/{bestConfig['configFile']}.yaml"
with open(bestConfigFilePath, "r") as f:
  cfg = yaml.load(f)
  bestConfig["transferRatio"] = cfg["ADAIN"]["LOADER"]["TRANSFER_RATIO"]
  bestConfig["alpha"] = cfg["ADAIN"]["MODEL"]["ALPHA"]

print(f"Best configuration:\n{bestConfig}")

env: VOC_ROOT=/content/drive/MyDrive/ProgettoML/datasets/VOC
env: CLIPART_ROOT=/content/drive/MyDrive/ProgettoML/datasets/CLIPART
rm: cannot remove 'outputs/vgg_ssd300_voc0712_AdaINst_ft_cv_1': No such file or directory
rm: cannot remove 'outputs/vgg_ssd300_voc0712_AdaINst_ft_cv_2': No such file or directory
rm: cannot remove 'outputs/vgg_ssd300_voc0712_AdaINst_ft_cv_3': No such file or directory
rm: cannot remove 'outputs/vgg_ssd300_voc0712_AdaINst_ft_cv_4': No such file or directory
DATASETS:
  STYLE: ('clipart_train',)
IMPL_FOLDER: AdaIN
INPUT:
  PRESERVE_COLOR: True
  STYLE_CROP: False
  STYLE_SIZE: 512
LOADER:
  TRANSFER_RATIO: 0.5
MODEL:
  ALPHA: 0.5
  DECODER: AdaIN/models/decoder.pth
  VGG: AdaIN/models/vgg_normalized.pth
2021-04-29 09:54:16,727 SSD INFO: Using 1 GPUs
2021-04-29 09:54:16,727 SSD INFO: Namespace(config_file='configs/vgg_ssd300_voc0712_AdaINst_ft_cv_1.yaml', distributed=False, enable_style_transfer=True, eval_step=2500, local_rank=0, log_step=10, num_gpus=1, opts

FileNotFoundError: ignored

## Retraining with the best obtained model

In [ ]:
os.chdir(os.environ["SSD_FOLDER"])

lastCheckpointFilePath = "outputs/vgg_ssd300_voc0712/last_checkpoint.txt"
configFile = "vgg_ssd300_voc0712_AdaINst_ft5"
GUARD = True

if not GUARD:
  folderPath = f"outputs/{configFile}"
  !rm -r $folderPath
  !mkdir $folderPath
  !cp $lastCheckpointFilePath $folderPath/
  
  configFilePath = f"configs/{configFile}.yaml"
  !python train.py --config-file $configFilePath --enable_style_transfer
  
resultsPath = f"outputs/{configFile}/inference/clipart_test/*.txt"
for filename in glob.glob(resultsPath):
  with open(filename, 'r') as f:
    line = f.read()
    mAP = float(line.split()[1])
print(f"Obtained mAP: {mAP}")

Obtained mAP: 0.3854
